In [ ]:
import os


!pip install keras4torch
!pip install einops



os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import keras4torch
from   keras4torch.callbacks  import ModelCheckpoint,LRScheduler
import torch
import torch.nn    as nn
import torch.optim as optim
import  torch.nn.functional as     F
import numpy       as np
import pandas      as pd
from copy import deepcopy
import  matplotlib.pyplot   as     plt
from    sklearn.preprocessing import StandardScaler, QuantileTransformer
from    datetime import datetime
import  gc
import STab
from STab import MyClassLoss,MyRegreLoss,CatMap,Num_Cat
from   STab import mainmodel, LWTA, Gsoftmax
MainModel=mainmodel.MainModel

In [2]:
##Split the data # Vale ta opos ta references
X_test=pd.DataFrame(np.load('Data/otto/X_num_test.npy')).astype(np.float64).values
y_test=pd.DataFrame(np.load('Data/otto/Y_test.npy')).astype(np.int32)[0]

X_train=pd.DataFrame(np.load('Data/otto/X_num_train.npy')).astype(np.float64).values
Y_train=pd.DataFrame(np.load('Data/otto/Y_train.npy')).astype(np.int32)[0]


X_valid=pd.DataFrame(np.load('Data/otto/X_num_val.npy')).astype(np.float64).values
y_valid=pd.DataFrame(np.load('Data/otto/Y_val.npy')).astype(np.int32)[0]

In [3]:
def True_ACC(p,t):
                return np.mean(np.where(np.round(p)==np.round(t),1,0))

In [4]:
chpfilename='saved_vanilla/savefileOT'

In [5]:
Or_model = MainModel(
    categories        = (),
    num_continuous    = 93,
    dim               = 192,
    dim_out           = 9,
    depth             = 5,
    heads             = 8,
    attn_dropout      = 0.25,
    ff_dropout        = 0.25,
    U                 = 2,
    cases             = 16

)

no_model = Num_Cat(Or_model,num_number=93,classes=9,Sample_size=16)
model    = keras4torch.Model(no_model,).build([93])

#Warm Up Train
optimizer=torch.optim.AdamW(model.parameters(),lr=0.0001,weight_decay=0.0001,)
sch=torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.00001, total_iters=5,  verbose=False)
model.compile(optimizer=optimizer, loss=MyClassLoss(0.1,1), metrics=['accuracy'])
callbacks=[LRScheduler(sch)]
model.fit(X_train, Y_train.values,
        epochs=10, batch_size=512,
        validation_data=(X_valid,y_valid.values),
        verbose=2,validation_batch_size=128,
        callbacks=callbacks)


#Main Train
optimizer=torch.optim.AdamW(model.parameters(),lr=0.001,weight_decay=0.0001,)
model.compile(optimizer=optimizer, loss=MyClassLoss(0.01,1), metrics=['accuracy', F.cross_entropy])
scheduler =LRScheduler( torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=7, factor=0.5,min_lr=0.00001))
callbacks=[scheduler,ModelCheckpoint(chpfilename,monitor='val_acc',mode='max')]
model.fit(X_train, Y_train.values,
        epochs=90, batch_size=512,
        validation_data=(X_valid,y_valid.values),
        verbose=2,validation_batch_size=128,
        callbacks=callbacks)


model.load_weights(chpfilename)
no_model.reset_Sample_size(1)

logits=0
for i in range(0,64):

    logits+=pd.DataFrame(model.predict(X_test,batch_size=512))

Test = (True_ACC(logits.idxmax(axis=1).values.reshape((-1,1)),y_test.values.reshape((-1,1))))

print('Test: ',Test)

[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert int32 to int64, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert int32 to int64, this could lead to extra memory usage.
Train on 39601 samples, validate on 9901 samples:
Epoch 1/10 - 155s - loss: 2.1519 - acc: 0.1190 - val_loss: 2.0475 - val_acc: 0.1327 - lr: 1e-09
Epoch 2/10 - 161s - loss: 1.5367 - acc: 0.4597 - val_loss: 1.2249 - val_acc: 0.6296 - lr: 2e-05
Epoch 3/10 - 162s - loss: 1.0385 - acc: 0.6483 - val_loss: 0.8870 - val_acc: 0.7151 - lr: 4e-05
Epoch 4/10 - 162s - loss: 0.8503 - acc: 0.6949 - val_loss: 0.7598 - val_acc: 0.7349 - lr: 6e-05
Epoch 5/10 - 161s - loss: 0.7767 - acc: 0.7125 - val_loss: 0.7011 - val_acc: 0.7463 - lr: 8e-05
Epoch 6/10 - 163s - loss: 0.7330 - acc: 0.7242 - val_loss: 0.6669 - val_acc: 0.7516 - lr: 1e-04
Epoch 7/10 - 162s - loss: 0.7058 - acc: 0.7326 - val_l

In [6]:
Test = (True_ACC(logits.idxmax(axis=1).values.reshape((-1,1)),y_test.values.reshape((-1,1))))

print('Test: ',Test)

Test:  0.8260342598577892
